In [30]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting  import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

In [ ]:
def time_split(df, train_size = .5, validate_size = .3):
    '''Splits time series data based on percentages and returns train, validate, test THE
    DATAFRAME MUST BE CHRONOLOGICALLY SORTED!'''
    t_size = int(len(df) * train_size)
    v_size = int(len(df) * validate_size)
    end = t_size + v_size
    return df[0:t_size], df[t_size:end], df[end:len(df)+1]

def plot_samples(target_var):
    '''
   plot each attribute 
   '''
    plt.figure(figsize = (12,4))
    sns.lineplot(data=train[target_var], label='train')
    sns.lineplot(data=validate[target_var], label='validate')
    sns.lineplot(data=test[target_var], label='test')
    plt.title(target_var.title())
    plt.legend()
    
def evaluate(target_var):
    '''
    the evaluate function will take in the actual values and the predicted values
    and compute the mean_squared_error and then take the sqrt returning a rounded rmse
    '''
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])),0)
    return rmse

def plot_and_eval(target_var):
    '''
    a function to evaluate forecasts by computing the rmse and plot train and validate along with predictions
    '''
    plot_samples(target_var)
    sns.lineplot(data=yhat_df[target_var], label='RMSE')
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, f'--RMSE: {rmse:.0f}')
    plt.show()

def append_eval_df(model_type, target_var):
    '''
    this function will take in the model type as a string, target variable
    as a string, and run the evaluate function to compute rmse, 
    and append to the dataframe a row with the model type, 
    target variable and rmse. 
    '''
    rmse = evaluate(target_var)
    d= {'model_type':[model_type], 'target_var':[target_var], 'rmse':[rmse]}
    d= pd.DataFrame(d)
    return eval_df.append(d, ignore_index= True)

def previous_plot_and_eval(target_var):
    '''
    a function to evaluate forecasts by computing the rmse and plot train and validate along with predictions
    '''
    rmse = evaluate(target_var)
    print(target_var, f'--RMSE: {rmse:.0f}')
    return rmse

In [2]:
df = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
df.Timestamp = pd.to_datetime(df.Timestamp, unit='s')b

In [10]:
df[df.Open.isna()]

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
1,2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011-12-31 07:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4856245,2021-03-30 05:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856308,2021-03-30 06:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856376,2021-03-30 07:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4857340,2021-03-30 23:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df[df.Close.isna()].set_index('Timestamp').resample('2000H').min()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2011-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-23 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-14 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-11-28 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-02-19 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-08-05 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-27 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
